In [1]:
from stickerChartDefinition import * 
BC.run(arr = True)

In [ ]:
str(list(BC.FBs[1].Sources.ins[1].datasource["sdata"].keys()))[1:-1]

In [ ]:
BC.run(arr = True)

In [2]:
from graphtools import *

In [3]:
def BCToNodes(BC,scale=1,marginal=10):
    nodes=[]
    nodegroups=[]
    for FB in BC.FBs:
        sx=FB.size[0]+marginal
        x=FB.gpos[0]+sx/2
        sy=FB.size[1]+marginal
        y=FB.gpos[1]+sy/2
        sx*=scale
        sy*=scale
        x*=scale
        y*=scale
        n=node(x,y,sx,sy)
        connFBs=[]
        for i in FB.Sources.ins:
            connFBs.append(i.datasource["source"].parent)
        for i in FB.Sources.outs:
            connFBs.append(i.datasource["source"].parent)
        n.cargo=[FB,connFBs,[]]
        nodes.append(n)
        if n.cargo[0].function.type =="SubChart":
            nodes.pop()
            scnodes, scnodegroups = BCToNodes(n.cargo[0].function.chart,scale)
            for sn in scnodes:
                nodes.append(sn)
            for sg in scnodegroups:
                nodegroups.append(sg)
            scnodegroups[-1].cargo=[FB,connFBs,[]]
            nodes.append(scnodegroups[-1])
    ng=nodegroup(1,2,3,4,nodes.copy())
    ng.BoundingBoxSet(resize=True)
    nodegroups.append(ng)
    
    
    if BC.minimized:
        sx=BC.parent.parent.size[0]+marginal
        x=BC.parent.parent.gpos[0]+sx/2
        sy=BC.parent.parent.size[1]+marginal
        y=BC.parent.parent.gpos[1]+sy/2
        sx*=scale
        sy*=scale
        x*=scale
        y*=scale

        BCnode=node(x,y,sx,sy)
        BCnodeconn=[]
        BCnodealias =[]
        for n in nodes:
            BCnodeconn =BCnodeconn + n.cargo[1]
            BCnodealias.append(n.cargo[0])
        BCnode.cargo=[BC.parent.parent,BCnodeconn,BCnodealias]
        nodes = [BCnode]
        nodegroups =[]
    
    return nodes, nodegroups

def connBCnodes(nodes):
    i=0
    for n in nodes:
        i+=1
        for nn in nodes:
            for nc in n.cargo[1]:
                if nc == nn.cargo[0]:
                    n.conn.append(nn)
                for nna in nn.cargo[2]: #check aliases
                    if nc == nna:
                        n.conn.append(nn)

def cleanNodegroups(nodegroups):
    i=0
    for g in nodegroups:
        i+=1
        for gg in nodegroups[i:]:
            for n in g.nodes:
                for nn in gg.nodes:
                    if n==nn:
                        gg.nodes.remove(nn)
        for n in g.nodes:
            if type(n)==type(g):
                n.parent=g

def shuffleNodes(nodes, xmax,ymax):
    for n in nodes:
        n.x=np.random.random()*xmax
        n.y=np.random.random()*ymax
                
                        
def BCToGraph(BC,scale=1):
    nodes, nodegroups = BCToNodes(BC,scale)
    connBCnodes(nodes)
    cleanNodegroups(nodegroups)
    GR = graph(nodes,nodegroups)
    return GR

def GraphValuesToBC(GR,scale=1,marginal=10):
    for g in GR.nodegroups:
        for n in g.nodes:
            xg=int((n.x-n.sx)/scale)
            yg=int((n.y-n.sy)/scale)
            x=xg-int((g.x-g.sx)/scale)
            y=yg-int((g.y-g.sy)/scale)
            sizex=int(2*n.sx/scale)-marginal
            sizey=int(2*n.sy/scale)-marginal
            n.cargo[0].gpos=np.array([xg,yg])
            n.cargo[0].pos=np.array([x,y])            
            n.cargo[0].size=np.array([sizex,sizey])
        

In [5]:
scale=0.4

GR = BCToGraph(BC,scale)
shuffleNodes(GR.nodes,1000,1000)
cv2.startWindowThread()

for g in GR.nodegroups:
    g.BoundingBoxSet(resize=True)
#GR.collapseGraph()
img = GR.DrawGraph()
cv2.imshow("Graafi1",img)
inp= cv2.waitKey(1)

while True:
    GR.StepPotNodePoints()
    #GR.StepPotNodegroupsPoints()
    img = GR.DrawGraph()
    cv2.imshow("Graafi",img)
    inp= cv2.waitKey(1)
    if inp==27:
        break
'''
while True:
    GR.StepPotNodePoints()
    GR.StepPotNodegroupsPoints()
    img = GR.DrawGraph()
    cv2.imshow("Graafi",img)
    inp= cv2.waitKey(1)
    if inp==27:
        break

while True:
    GR.MoveAll(ngs=True)#,gps_in=True)
    img = GR.DrawGraph()
    cv2.imshow("Graafi",img)
    inp= cv2.waitKey(1)
    if inp==27:
        break

sc=0.1
sc2=1
while True:
    sc2*=0.99
    sc2 =max(sc2,.01)
    GR.MoveAll(sc2=.5,sc=sc2)#,gps_in=True)
    img = GR.DrawGraph()
    cv2.imshow("Graafi",img)
    inp= cv2.waitKey(1)
    if inp==27:
        break
'''     
GR.collapseGraph()
GR.MovenodesWithGroups()
img = GR.DrawGraph()
cv2.imshow("Graafi2",img)
inp= cv2.waitKey(3000)
GraphValuesToBC(GR,scale)
BC.run(arr=False)

In [ ]:
GraphValuesToBC(GR,scale)
BC.run(arr=False)

In [ ]:
'''
cv2.startWindowThread()
#while True:
#nodegroups[0].BoundingBoxSet(resize=True)
img = GR.DrawGraph()
cv2.imshow("Graafi",img)
inp= cv2.waitKey(10000)
#for i in range(len(nodegroups[0].nodes)):
#GR.nodegroups[-1].collapse(np.array([-1,-1]))
GR.collapseGraph()
GR.MovenodesWithGroups()
img = GR.DrawGraph()
cv2.imshow("Graafi2",img)
while True:
    inp= cv2.waitKey(3000)
    if inp==27:
        break
'''
cv2.destroyAllWindows()
cv2.waitKey(1)
